# Create model with BigML and first submission to Kaggle

In [1]:
from pandas import read_csv
import bigml.api
from bigml.api import BigML
api= BigML (project='project/5d8dbde0eba31d7fc0000865')

source = api.create_source('train.csv')

In [2]:
api.ok(source)

True

In [3]:
dataset_trainfull = api.create_dataset(source)

In [4]:
api.ok(dataset_trainfull)

True

In [5]:
#Split on trainfull 80/20
train_dataset = api.create_dataset(
    dataset_trainfull, {"name": "Train80",
                     "sample_rate": 0.8, "seed": "my seed"})
val = api.create_dataset(
    dataset_trainfull, {"name": "Train20",
                     "sample_rate": 0.8, "seed": "my seed",
                     "out_of_bag": True})

In [6]:
api.ok(train_dataset)
api.ok(val)

True

In [8]:
# Deepnet
model = api.create_deepnet(train_dataset, {"objective_field":"SalePrice"})
api.ok(model)

True

In [9]:
evaluation = api.create_evaluation(model, train_dataset)
api.ok(evaluation)

True

In [10]:
api.pprint(evaluation['object']['result'])

{   'mean': {   'mean_absolute_error': 55557.1555,
                'mean_squared_error': 5837936730.33284,
                'r_squared': 0},
    'model': {   'mean_absolute_error': 13481.46002,
                 'mean_squared_error': 427736078.70259,
                 'per_class_statistics': [],
                 'r_squared': 0.92673},
    'random': {   'mean_absolute_error': 251162.43219,
                  'mean_squared_error': 95566842055.86018,
                  'r_squared': -15.36997}}


In [11]:
# linear regression
linear_regression = api.create_linear_regression(train_dataset)

api.ok(linear_regression)

True

In [12]:
evaluation_lr = api.create_evaluation(linear_regression, train_dataset)
api.ok(evaluation_lr)
api.pprint(evaluation_lr['object']['result'])

{   'mean': {   'mean_absolute_error': 55557.1555,
                'mean_squared_error': 5837936730.33284,
                'r_squared': 0},
    'model': {   'mean_absolute_error': 4982790865516.127,
                 'mean_squared_error': 2.4898650086353646e+25,
                 'per_class_statistics': [],
                 'r_squared': -4264974294254487},
    'random': {   'mean_absolute_error': 255583.57177,
                  'mean_squared_error': 97571737818.6484,
                  'r_squared': -15.71339}}


In [13]:
#Prediction on deepnet model
batch_prediction = api.create_batch_prediction(model, val, {
    "all_fields": True, "confidence": True, "probabilities": True, "prediction_name": "my_prediction"})
api.ok(batch_prediction)

True

In [22]:
api.download_batch_prediction(batch_prediction,
    filename='prediction_val.csv')

'prediction_val.csv'

In [24]:
prediction_val = read_csv('prediction_val.csv', index_col=0)

Calculation of % Error

In [25]:
for row in [prediction_val]:
    row['% Error'] = ((row['my_prediction'] - row['SalePrice'])/row['SalePrice'])*100
row

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,my_prediction,% Error
Id,,,,,,,,,,,,,,,,,,,,,
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,NaN,NaN,0,12,2008,WD,Normal,250000,308191.09949,23.276440
12,60,RL,85.0,11924,Pave,NaN,IR1,Lvl,AllPub,Inside,...,NaN,NaN,0,7,2006,New,Partial,345000,387867.84512,12.425462
15,20,RL,NaN,10920,Pave,NaN,IR1,Lvl,AllPub,Corner,...,GdWo,NaN,0,5,2008,WD,Normal,157000,145697.54269,-7.199017
16,45,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Corner,...,GdPrv,NaN,0,7,2007,WD,Normal,132000,128303.45780,-2.800411
19,20,RL,66.0,13695,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,NaN,0,6,2008,WD,Normal,159000,146930.18731,-7.591077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,30,RL,NaN,8854,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,NaN,0,5,2009,WD,Normal,121000,102943.99865,-14.922315
1448,60,RL,80.0,10000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,NaN,0,12,2007,WD,Normal,240000,247136.67949,2.973616
1453,180,RM,35.0,3675,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,NaN,0,5,2006,WD,Normal,145000,132150.85288,-8.861481


Calculation of Error

In [26]:
for row in [prediction_val]:
    row['Error'] = row['my_prediction'] - row['SalePrice']
row

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,my_prediction,% Error,Error
Id,,,,,,,,,,,,,,,,,,,,,
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,NaN,0,12,2008,WD,Normal,250000,308191.09949,23.276440,58191.09949
12,60,RL,85.0,11924,Pave,NaN,IR1,Lvl,AllPub,Inside,...,NaN,0,7,2006,New,Partial,345000,387867.84512,12.425462,42867.84512
15,20,RL,NaN,10920,Pave,NaN,IR1,Lvl,AllPub,Corner,...,NaN,0,5,2008,WD,Normal,157000,145697.54269,-7.199017,-11302.45731
16,45,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Corner,...,NaN,0,7,2007,WD,Normal,132000,128303.45780,-2.800411,-3696.54220
19,20,RL,66.0,13695,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,0,6,2008,WD,Normal,159000,146930.18731,-7.591077,-12069.81269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,30,RL,NaN,8854,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,0,5,2009,WD,Normal,121000,102943.99865,-14.922315,-18056.00135
1448,60,RL,80.0,10000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,0,12,2007,WD,Normal,240000,247136.67949,2.973616,7136.67949
1453,180,RM,35.0,3675,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,0,5,2006,WD,Normal,145000,132150.85288,-8.861481,-12849.14712


Calculation of Absolute Error

In [27]:
for row in [prediction_val]:
    row['Error_ABS'] = abs(row['Error'])
row

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,my_prediction,% Error,Error,Error_ABS
Id,,,,,,,,,,,,,,,,,,,,,
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,12,2008,WD,Normal,250000,308191.09949,23.276440,58191.09949,58191.09949
12,60,RL,85.0,11924,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,7,2006,New,Partial,345000,387867.84512,12.425462,42867.84512,42867.84512
15,20,RL,NaN,10920,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,5,2008,WD,Normal,157000,145697.54269,-7.199017,-11302.45731,11302.45731
16,45,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Corner,...,0,7,2007,WD,Normal,132000,128303.45780,-2.800411,-3696.54220,3696.54220
19,20,RL,66.0,13695,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,6,2008,WD,Normal,159000,146930.18731,-7.591077,-12069.81269,12069.81269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,30,RL,NaN,8854,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,5,2009,WD,Normal,121000,102943.99865,-14.922315,-18056.00135,18056.00135
1448,60,RL,80.0,10000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,12,2007,WD,Normal,240000,247136.67949,2.973616,7136.67949,7136.67949
1453,180,RM,35.0,3675,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,5,2006,WD,Normal,145000,132150.85288,-8.861481,-12849.14712,12849.14712


MAE =  mean absolute error

In [28]:
import numpy as np

target_column = 'Error_ABS'
a = prediction_val[target_column].values

MAE = np.mean(a)

MAE

18967.60566921233

In [29]:
for row in [prediction_val]:
    row['Error_ABS_squared'] = row['Error']*row['Error']
row

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,MoSold,YrSold,SaleType,SaleCondition,SalePrice,my_prediction,% Error,Error,Error_ABS,Error_ABS_squared
Id,,,,,,,,,,,,,,,,,,,,,
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,12,2008,WD,Normal,250000,308191.09949,23.276440,58191.09949,58191.09949,3.386204e+09
12,60,RL,85.0,11924,Pave,NaN,IR1,Lvl,AllPub,Inside,...,7,2006,New,Partial,345000,387867.84512,12.425462,42867.84512,42867.84512,1.837652e+09
15,20,RL,NaN,10920,Pave,NaN,IR1,Lvl,AllPub,Corner,...,5,2008,WD,Normal,157000,145697.54269,-7.199017,-11302.45731,11302.45731,1.277455e+08
16,45,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Corner,...,7,2007,WD,Normal,132000,128303.45780,-2.800411,-3696.54220,3696.54220,1.366442e+07
19,20,RL,66.0,13695,Pave,NaN,Reg,Lvl,AllPub,Inside,...,6,2008,WD,Normal,159000,146930.18731,-7.591077,-12069.81269,12069.81269,1.456804e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,30,RL,NaN,8854,Pave,NaN,Reg,Lvl,AllPub,Inside,...,5,2009,WD,Normal,121000,102943.99865,-14.922315,-18056.00135,18056.00135,3.260192e+08
1448,60,RL,80.0,10000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,12,2007,WD,Normal,240000,247136.67949,2.973616,7136.67949,7136.67949,5.093219e+07
1453,180,RM,35.0,3675,Pave,NaN,Reg,Lvl,AllPub,Inside,...,5,2006,WD,Normal,145000,132150.85288,-8.861481,-12849.14712,12849.14712,1.651006e+08


MSE = mean squared error

In [30]:
target_column = 'Error_ABS_squared'
b = prediction_val[target_column].values

MSE = np.mean(b)

print(MSE)

1205403955.6224344


In [31]:
export_csv = prediction_val.to_csv ('prediction_error.csv', header=True)

In [32]:
#Upload the test
test_source = api.create_source('test.csv')
api.ok(test_source)

True

In [33]:
dataset_test = api.create_dataset(test_source)
api.ok(dataset_test)

True

In [34]:
# Deepnet on the trainfull dataset
model_2 = api.create_ensemble(dataset_trainfull, {"objective_field":"SalePrice"})
api.ok(model_2)

True

In [35]:
batch_prediction_full = api.create_batch_prediction(model_2, dataset_test, {"all_fields": True, "confidence": True, "probabilities": True, "prediction_name": "my_prediction"})
api.ok(batch_prediction_full)

True

In [36]:
evaluation_full = api.create_evaluation(model_2, dataset_trainfull)
api.ok(evaluation_full)
api.pprint(evaluation_full['object']['result'])

{   'mean': {   'mean_absolute_error': 57434.77028,
                'mean_squared_error': 6306788585.3493,
                'r_squared': 0},
    'model': {   'mean_absolute_error': 9303.38198,
                 'mean_squared_error': 226804885.56528,
                 'per_class_statistics': [],
                 'r_squared': 0.96404},
    'random': {   'mean_absolute_error': 251836.55048,
                  'mean_squared_error': 94384820921.42238,
                  'r_squared': -13.96559}}


In [37]:
api.download_batch_prediction(batch_prediction_full,
    filename='prediction_test.csv')

'prediction_test.csv'

In [38]:
# Import the new prediction and remove the columns/replace Id and SalePrice
import pandas as pd
df = pd.read_csv('prediction_test.csv')
shape = df.shape
print(shape)

(1459, 82)


In [39]:
list(df.columns.values)

['Id',
 'MSSubClass',
 'MSZoning',
 'LotFrontage',
 'LotArea',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF'

In [42]:
id_col = df['Id'].values.tolist()
prediction_col = df['my_prediction'].values.tolist()

In [43]:
import pandas as pd
submission = pd.DataFrame()
submission['Id'] = id_col
submission['SalePrice'] = prediction_col

In [44]:
submission.to_csv('submission_1.csv', index=False)

In [45]:
import kaggle
kaggle.api.competition_submit("submission_1.csv", "test", "house-prices-advanced-regression-techniques")

100%|██████████| 27.5k/27.5k [00:03<00:00, 8.32kB/s]


Successfully submitted to House Prices: Advanced Regression Techniques

In [46]:
score = 0.15452
print("Kaggle Score:", score)

Kaggle Score: 0.15452
